# Dataset Upload

In addition to creating and editing Datasets in the LangSmith UI, you can also create and edit datasets with the LangSmith SDK.

Let's go ahead an upload a list of examples that we have from our RAG application to LangSmith as a new dataset.

In [ ]:
import os



os.environ["GROQ_API_KEY"] = ""   # ✅ Use Groq instead
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"


In [4]:
# from openai import OpenAI   ❌ Remove
from groq import Groq   # ✅ Use Groq SDK

groq_client = Groq(api_key=os.environ["GROQ_API_KEY"])


In [21]:
import os
from groq import Groq
from langsmith import traceable

# Initialize Groq client
groq_client = Groq(api_key=os.environ["GROQ_API_KEY"])

# Traceable ensures runs are logged to LangSmith
@traceable
def langsmith_rag(question: str) -> str:
    """
    Simple RAG-style call using Groq as the LLM.
    """

    # You can extend this by adding retrieved context before the question
    messages = [
        {"role": "system", "content": "You are a helpful assistant that answers questions clearly."},
        {"role": "user", "content": question},
    ]

    response = groq_client.chat.completions.create(
        model="openai/gpt-oss-120b",   # Groq model (fast + cost-efficient)
        messages=messages,
    )

    answer = response.choices[0].message.content   # ✅ correct

    return answer


In [6]:
from langsmith import Client

example_inputs = [
("How do I set up tracing to LangSmith if I'm using LangChain?", "To set up tracing to LangSmith while using LangChain, you need to set the environment variable `LANGSMITH_TRACING` to 'true'. Additionally, you must set the `LANGSMITH_API_KEY` environment variable to your API key. By default, traces will be logged to a project named \"default.\""),
("How can I trace with the @traceable decorator?", "To trace with the @traceable decorator in Python, simply decorate any function you want to log traces for by adding `@traceable` above the function definition. Ensure that the LANGSMITH_TRACING environment variable is set to 'true' to enable tracing, and also set the LANGSMITH_API_KEY environment variable with your API key. By default, traces will be logged to a project named \"default,\" but you can configure it to log to a different project if needed."),
("How do I pass metadata in with @traceable?", "You can pass metadata with the @traceable decorator by specifying arbitrary key-value pairs as arguments. This allows you to associate additional information, such as the execution environment or user details, with your traces. For more detailed instructions, refer to the LangSmith documentation on adding metadata and tags."),
("What is LangSmith used for in three sentences?", "LangSmith is a platform designed for the development, monitoring, and testing of LLM applications. It enables users to collect and analyze unstructured data, debug issues, and create datasets for testing and evaluation. The tool supports various workflows throughout the application development lifecycle, enhancing the overall performance and reliability of LLM applications."),
("What testing capabilities does LangSmith have?", "LangSmith offers capabilities for creating datasets of inputs and reference outputs to run tests on LLM applications, supporting a test-driven approach. It allows for bulk uploads of test cases, on-the-fly creation, and exporting from application traces. Additionally, LangSmith facilitates custom evaluations to score test results, enhancing the testing process."),
("Does LangSmith support online evaluation?", "Yes, LangSmith supports online evaluation as a feature. It allows you to configure a sample of runs from production to be evaluated, providing feedback on those runs. You can use either custom code or an LLM as a judge for the evaluations."),
("Does LangSmith support offline evaluation?", "Yes, LangSmith supports offline evaluation through its evaluation how-to guides and features for managing datasets. Users can manage datasets for offline evaluations and run various types of evaluations, including unit testing and auto-evaluation. This allows for comprehensive testing and improvement of LLM applications."),
("Can LangSmith be used for finetuning and model training?", "Yes, LangSmith can be used for fine-tuning and model training. It allows you to capture run traces from your deployment, query and filter this data, and convert it into a format suitable for fine-tuning models. Additionally, you can create training datasets to keep track of the data used for model training."),
("Can LangSmith be used to evaluate agents?", "Yes, LangSmith can be used to evaluate agents. It provides various evaluation strategies, including assessing the agent's final response, evaluating individual steps, and analyzing the trajectory of tool calls. These methods help ensure the effectiveness of LLM applications."),
("How do I create user feedback with the LangSmith sdk?", "To create user feedback with the LangSmith SDK, you first need to run your application and obtain the `run_id`. Then, you can use the `create_feedback` method, providing the `run_id`, a feedback key, a score, and an optional comment. For example, in Python, it would look like this: `client.create_feedback(run_id, key=\"feedback-key\", score=1.0, comment=\"comment\")`."),
]

client = Client()
# TODO: Fill in dataset id
dataset_id = "ecf95159-4a0b-4e91-9087-c4f048dec756"

# Prepare inputs and outputs for bulk creation
inputs = [{"question": input_prompt} for input_prompt, _ in example_inputs]
outputs = [{"output": output_answer} for _, output_answer in example_inputs]

client.create_examples(
  inputs=inputs,
  outputs=outputs,
  dataset_name="Golden Dataset",
)

{'example_ids': ['3f5af433-5a4e-4e61-a355-f6ac78d98a7c',
  '77eb4d8d-5621-46f6-be4e-3fa8c235d0eb',
  '8ae3fc92-2b18-4ea9-ba01-ed1618cc5e3f',
  '4b66882f-a29c-4d72-8419-120270bed93a',
  '225bc0a7-9c8f-4e61-925a-7e974bd34780',
  '2c53c635-bf6d-427b-b1a6-0812fd1875b3',
  '62d9d7e2-2b18-4a24-a159-9f64bdbb3d20',
  'c8ad1754-24d9-4da4-bbe4-5fe3460d0b46',
  '3aeef2d3-aaaa-41ad-bf5f-bafdc8eff56a',
  '88b214f4-9110-483f-ae41-aaf2db04ea04'],
 'count': 10}

## Submitting another Trace

I've moved our RAG application definition to `app.py` so we can quickly import it.

In [ ]:


# Ask a question, this will:
# 1. Retrieve context from vector DB (Groq embeddings)
# 2. Call Groq LLM (model you set)
# 3. Trace everything into LangSmith
question = "How do I set up tracing to LangSmith if I'm using LangChain?"
answer = langsmith_rag(question)

print("Answer:", answer)


Answer: Below is a **step‑by‑step guide** for wiring LangChain (Python) to **LangSmith** so that every LLM call, chain run, tool use, and agent step shows up in your LangSmith dashboard.

---

## 1️⃣ Install the required packages

```bash
# Core LangChain + LLM provider (e.g. OpenAI)
pip install langchain openai

# LangSmith SDK (includes the callback handler)
pip install langsmith
```

> **Tip:** If you are already using `langchain[all]` you’ll get `langsmith` automatically, but it never hurts to run the explicit install.

---

## 2️⃣ Get your LangSmith credentials

1. Log in to https://smith.langchain.com (or your self‑hosted instance).  
2. Click **Settings → API Keys** → **Create new key**.  
3. Copy the **API key** (e.g. `sk-xxxxxx`).  

You’ll also need a **project name** (or you can let LangSmith create one on‑the‑fly).

---

## 3️⃣ Configure the environment (recommended)

Set the variables **once** in your environment (Docker, `.env`, CI, etc.):

```bash
export LANGCHAIN_API_KE

In [3]:
%pip install -U langchain-openai